# dev_phase测试

In [1]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
use_cuda = torch.cuda.is_available()

Sat Nov  8 09:50:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.02                 Driver Version: 576.02         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   39C    P8             28W /  350W |   23802MiB /  24576MiB |      8%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
SEED = 42
WEIGHTED = True

In [3]:
from datasets import *
import pandas as pd
test = pd.DataFrame(load_dataset("dev_phase/subtask2/dev/eng", split="train"))

C:\Users\8888\Anaconda3\envs\SemEval2025_Task11\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
labels = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]
def get_text_and_label(df):  
  return df.rename(columns={"text": "text"})[["id", "text"]]
test = get_text_and_label(test)
print("Test split size:", len(test.index))
test.head()

Test split size: 133


,id,text
0,en_f66ca14d60851371f9720aaf4ccd9b58,God is with Ukraine and Zelensky
1,en_3a489aa7fed9726aa8d3d4fe74c57efb,"4 Dems, 2 Republicans Luzerne County Council s..."
2,en_95770ff547ea5e48b0be00f385986483,Abuse Survivor Recounts Her Struggles at YWCA ...
3,en_2048ae6f9aa261c48e6d777bcc5b38bf,"After Rwanda, another deportation camp disaster"
4,en_07781aa88e61e7c0a996abd1e5ea3a20,Another plea in Trump election interference probe


In [5]:
# EXP = "lr-2e-05-optimizer-AdamW-epochs-10-RMSEduringTraining"
EXP = "07-11-2025-17-22"
# TRANSFORMERS = "top_transformers"  
TRANSFORMERS = "07-11-2025-17-22" 

In [6]:
import os
PATH = "./outputs/"
EXP_PATH = os.path.join(PATH, EXP)
TRANSFORMERS_PATH = os.path.join(PATH, TRANSFORMERS)
print("Current working dir:", EXP_PATH)
OUTPUT = os.path.join(EXP_PATH, "test")
# 第一次使用时创建文件夹
# os.mkdir(EXP_PATH)
os.mkdir(OUTPUT)

Current working dir: ./outputs/07-11-2025-17-22


In [7]:
models = {    
"mbert-cased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-cased"    
},    
"mbert-uncased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-uncased"    
},    
"roberta": {        
"model_type": "roberta",        
"model_name": "models--FacebookAI--roberta-base"    
},    
"beto-cased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-cased"    
},    
"beto-uncased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-uncased"    
},    
"distilbert-multi": {        
"model_type": "distilbert",        
"model_name": "models--distilbert--distilbert-base-multilingual-cased"    
}
}

In [8]:
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs
for model, fields in models.items():    
  print("model:",model)
  models[model] = MultiLabelClassificationModel(fields["model_type"], os.path.join(TRANSFORMERS_PATH, model), use_cuda=False)

model: mbert-cased
model: mbert-uncased
model: roberta
model: beto-cased
model: beto-uncased
model: distilbert-multi


In [9]:
import json
best_ensemble = {}
with open(os.path.join(EXP_PATH, "best-ensemble.json")) as json_file:
    best_ensemble = json.load(json_file)
print("----- BEST ENSEMBLE -----")
for field in ["name", "models", "metrics"]:
  print(f"{field}:", best_ensemble.get(field))

----- BEST ENSEMBLE -----
name: ensemble19
models: ['beto-cased', 'distilbert-multi']
metrics: {'accuracy': 0.95665, 'macro_f1': 0.88668, 'macro_precision': 0.95511, 'macro_recall': 0.83622, 'weighted_f1': 0.95303}


In [10]:
model_evaluation = {}
for model in best_ensemble.get("models"):  
  with open(os.path.join(os.path.join(EXP_PATH, model), "model-evaluation.json")) as json_file:
      model_evaluation[model] = json.load(json_file)

In [11]:
from sklearn.preprocessing import normalize
def vote(predictions, weighted=False, weights=None):
  votes = list()  
  for i in range(len(labels)):    
    curr_label_preds = [preds[i] for preds in predictions]    
    voting = sum(curr_label_preds * weights) if weighted else sum(curr_label_preds)/len(curr_label_preds)
    votes.append(0 if voting < 0.5 else 1)  
  return votes
test_predictions = list()
def predict_ensemble(ensemble_name, dataset_name, dataset, weighted=False):  
 for i in range(len(dataset.index)):
    predictions = list()
    ensemble_models = best_ensemble.get("models")    
    for model_name in ensemble_models:
      curr_model_outputs = model_evaluation[model_name].get(f"{dataset_name}_model_outputs")
      predictions.append(curr_model_outputs[i])
    weights = list()    
    if weighted:      
      f1_scores_list = [model_evaluation[model_name]["metrics"].get("weighted_f1")                        
                        for model_name in best_ensemble.get("models")]
      weights = normalize([f1_scores_list], norm="l1")[0]
    ensemble_pred = vote(predictions, weighted, weights)
    test_predictions.append(ensemble_pred)

In [12]:
for model_name in best_ensemble.get("models"):
  model_predictions, model_raw_outputs = models.get(model_name).predict(test["text"].tolist())
  model_evaluation[model_name]["test_model_outputs"] = model_raw_outputs
  model_evaluation[model_name]["test_predictions"] = model_predictions
predict_ensemble(best_ensemble.get("name"), "test", test, weighted=WEIGHTED)
n = 5
print(f"First {n} predictions:", test_predictions[:n])

1it [00:02,  2.48s/it]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:09<00:00,  1.89it/s]
1it [00:02,  2.38s/it]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  3.72it/s]

First 5 predictions: [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [13]:
test_output = test[["id"]].rename(columns={"id": "id"})
for i in range(len(labels)):
  test_output[labels[i]] = [pred[i] for pred in test_predictions]
test_output.head(10)

,id,political,racial/ethnic,religious,gender/sexual,other
0,en_f66ca14d60851371f9720aaf4ccd9b58,0,0,0,0,0
1,en_3a489aa7fed9726aa8d3d4fe74c57efb,0,0,0,0,0
2,en_95770ff547ea5e48b0be00f385986483,0,0,0,0,0
3,en_2048ae6f9aa261c48e6d777bcc5b38bf,0,0,0,0,0
4,en_07781aa88e61e7c0a996abd1e5ea3a20,0,0,0,0,0
5,en_153d96f9dc27f0602c927223404d94b5,1,0,0,0,0
6,en_4ab5a4cc5c87d0af9cf4b80c301647bf,0,0,0,0,0
7,en_e75a95ba52930d6d72d503ab9469eb29,0,0,0,0,0
8,en_eb8fab668668f9959cafdecbfc0f081a,0,0,0,0,0
9,en_702724dc168d600e788d775c8e651f36,0,0,0,0,0


In [14]:
test_output.to_csv(os.path.join(OUTPUT, "results_test.csv"), index=False)

# dev_phase2 测试

In [15]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
use_cuda = torch.cuda.is_available()

Sat Nov  8 10:43:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.02                 Driver Version: 576.02         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   39C    P8             31W /  350W |   24061MiB /  24576MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
SEED = 42
WEIGHTED = True

In [17]:
from datasets import *
import pandas as pd
test = pd.DataFrame(load_dataset("dev_phase2/subtask2/dev/eng", split="train"))

Generating train split: 133 examples [00:00, 13023.66 examples/s]


In [18]:
labels = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]
def get_text_and_label(df):  
  return df.rename(columns={"text": "text"})[["id", "text"]]
test = get_text_and_label(test)
print("Test split size:", len(test.index))
test.head()

Test split size: 133


,id,text
0,eng_f66ca14d60851371f9720aaf4ccd9b58,God is with Ukraine and Zelensky
1,eng_3a489aa7fed9726aa8d3d4fe74c57efb,"4 Dems, 2 Republicans Luzerne County Council s..."
2,eng_95770ff547ea5e48b0be00f385986483,Abuse Survivor Recounts Her Struggles at YWCA ...
3,eng_2048ae6f9aa261c48e6d777bcc5b38bf,"After Rwanda, another deportation camp disaster"
4,eng_07781aa88e61e7c0a996abd1e5ea3a20,Another plea in Trump election interference probe


In [19]:
# EXP = "lr-2e-05-optimizer-AdamW-epochs-10-RMSEduringTraining"
EXP = "07-11-2025-17-22"
# TRANSFORMERS = "top_transformers"  
TRANSFORMERS = "07-11-2025-17-22" 

In [20]:
import os
PATH = "./outputs/"
EXP_PATH = os.path.join(PATH, EXP)
TRANSFORMERS_PATH = os.path.join(PATH, TRANSFORMERS)
print("Current working dir:", EXP_PATH)
OUTPUT = os.path.join(EXP_PATH, "test")
# 第一次使用时创建文件夹
# os.mkdir(EXP_PATH)
# os.mkdir(OUTPUT)

Current working dir: ./outputs/07-11-2025-17-22


In [21]:
models = {    
"mbert-cased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-cased"    
},    
"mbert-uncased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-uncased"    
},    
"roberta": {        
"model_type": "roberta",        
"model_name": "models--FacebookAI--roberta-base"    
},    
"beto-cased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-cased"    
},    
"beto-uncased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-uncased"    
},    
"distilbert-multi": {        
"model_type": "distilbert",        
"model_name": "models--distilbert--distilbert-base-multilingual-cased"    
}
}

In [22]:
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs
for model, fields in models.items():    
  print("model:",model)
  models[model] = MultiLabelClassificationModel(fields["model_type"], os.path.join(TRANSFORMERS_PATH, model), use_cuda=False)

model: mbert-cased
model: mbert-uncased
model: roberta
model: beto-cased
model: beto-uncased
model: distilbert-multi


In [23]:
import json
best_ensemble = {}
with open(os.path.join(EXP_PATH, "best-ensemble.json")) as json_file:
    best_ensemble = json.load(json_file)
print("----- BEST ENSEMBLE -----")
for field in ["name", "models", "metrics"]:
  print(f"{field}:", best_ensemble.get(field))

----- BEST ENSEMBLE -----
name: ensemble19
models: ['beto-cased', 'distilbert-multi']
metrics: {'accuracy': 0.95665, 'macro_f1': 0.88668, 'macro_precision': 0.95511, 'macro_recall': 0.83622, 'weighted_f1': 0.95303}


In [24]:
model_evaluation = {}
for model in best_ensemble.get("models"):  
  with open(os.path.join(os.path.join(EXP_PATH, model), "model-evaluation.json")) as json_file:
      model_evaluation[model] = json.load(json_file)

In [25]:
from sklearn.preprocessing import normalize
def vote(predictions, weighted=False, weights=None):
  votes = list()  
  for i in range(len(labels)):    
    curr_label_preds = [preds[i] for preds in predictions]    
    voting = sum(curr_label_preds * weights) if weighted else sum(curr_label_preds)/len(curr_label_preds)
    votes.append(0 if voting < 0.5 else 1)  
  return votes
test_predictions = list()
def predict_ensemble(ensemble_name, dataset_name, dataset, weighted=False):  
 for i in range(len(dataset.index)):
    predictions = list()
    ensemble_models = best_ensemble.get("models")    
    for model_name in ensemble_models:
      curr_model_outputs = model_evaluation[model_name].get(f"{dataset_name}_model_outputs")
      predictions.append(curr_model_outputs[i])
    weights = list()    
    if weighted:      
      f1_scores_list = [model_evaluation[model_name]["metrics"].get("weighted_f1")                        
                        for model_name in best_ensemble.get("models")]
      weights = normalize([f1_scores_list], norm="l1")[0]
    ensemble_pred = vote(predictions, weighted, weights)
    test_predictions.append(ensemble_pred)

In [26]:
for model_name in best_ensemble.get("models"):
  model_predictions, model_raw_outputs = models.get(model_name).predict(test["text"].tolist())
  model_evaluation[model_name]["test_model_outputs"] = model_raw_outputs
  model_evaluation[model_name]["test_predictions"] = model_predictions
predict_ensemble(best_ensemble.get("name"), "test", test, weighted=WEIGHTED)
n = 5
print(f"First {n} predictions:", test_predictions[:n])

1it [00:08,  8.56s/it]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:10<00:00,  1.57it/s]
1it [00:02,  2.40s/it]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  3.10it/s]

First 5 predictions: [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [27]:
test_output = test[["id"]].rename(columns={"id": "id"})
for i in range(len(labels)):
  test_output[labels[i]] = [pred[i] for pred in test_predictions]
test_output.head(10)

,id,political,racial/ethnic,religious,gender/sexual,other
0,eng_f66ca14d60851371f9720aaf4ccd9b58,0,0,0,0,0
1,eng_3a489aa7fed9726aa8d3d4fe74c57efb,0,0,0,0,0
2,eng_95770ff547ea5e48b0be00f385986483,0,0,0,0,0
3,eng_2048ae6f9aa261c48e6d777bcc5b38bf,0,0,0,0,0
4,eng_07781aa88e61e7c0a996abd1e5ea3a20,0,0,0,0,0
5,eng_153d96f9dc27f0602c927223404d94b5,1,0,0,0,0
6,eng_4ab5a4cc5c87d0af9cf4b80c301647bf,0,0,0,0,0
7,eng_e75a95ba52930d6d72d503ab9469eb29,0,0,0,0,0
8,eng_eb8fab668668f9959cafdecbfc0f081a,0,0,0,0,0
9,eng_702724dc168d600e788d775c8e651f36,0,0,0,0,0


In [28]:
test_output.to_csv(os.path.join(OUTPUT, "results_test2.csv"), index=False)

# test_phase 测试

In [1]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
use_cuda = torch.cuda.is_available()

Sun Feb  1 12:01:38 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.40                 Driver Version: 576.40         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   46C    P8             29W /  390W |   23610MiB /  24576MiB |     11%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
SEED = 42
WEIGHTED = True

In [5]:
from datasets import *
import pandas as pd
test = pd.DataFrame(load_dataset("test_phase/subtask2/test/eng", split="train"))

In [6]:
labels = ["political", "racial/ethnic", "religious", "gender/sexual", "other"]
def get_text_and_label(df):  
  return df.rename(columns={"text": "text"})[["id", "text"]]
test = get_text_and_label(test)
print("Test split size:", len(test.index))
test.head()

Test split size: 1452


,id,text
0,eng_0d46cf5c8e1f037159cfd6c9dcde03bb,OP! Podcast where you ascertain the immigratio...
1,eng_0676b42cc5e76f3a2b7cf3b562676fc1,"""...the latest sign that Israel is planning fo..."
2,eng_f3ce74f07a4cd9ee1926b87b40b78887,"""Ice discs, ice circles, ice pans, ice pancake..."
3,eng_9bb8d97251cc86cf5943c31be5f1b9a6,"""Since 2021, USAIDWest Bank and Gaza has inves..."
4,eng_755cacee0509beb4dc49f0fa1fd5d22f,"""The Cybersecurity Infrastructure Security Age..."


In [7]:
# EXP = "lr-2e-05-optimizer-AdamW-epochs-10-RMSEduringTraining"
EXP = "07-11-2025-17-22"
# TRANSFORMERS = "top_transformers"  
TRANSFORMERS = "07-11-2025-17-22" 

In [8]:
import os
PATH = "./outputs/"
EXP_PATH = os.path.join(PATH, EXP)
TRANSFORMERS_PATH = os.path.join(PATH, TRANSFORMERS)
print("Current working dir:", EXP_PATH)
OUTPUT = os.path.join(EXP_PATH, "test")
# 第一次使用时创建文件夹
# os.mkdir(EXP_PATH)
# os.mkdir(OUTPUT)

Current working dir: ./outputs/07-11-2025-17-22


In [9]:
models = {    
"mbert-cased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-cased"    
},    
"mbert-uncased": {        
"model_type": "bert",        
"model_name": "models--google-bert--bert-base-multilingual-uncased"    
},    
"roberta": {        
"model_type": "roberta",        
"model_name": "models--FacebookAI--roberta-base"    
},    
"beto-cased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-cased"    
},    
"beto-uncased": {        
"model_type": "bert",        
"model_name": "models--dccuchile--bert-base-spanish-wwm-uncased"    
},    
"distilbert-multi": {        
"model_type": "distilbert",        
"model_name": "models--distilbert--distilbert-base-multilingual-cased"    
}
}

In [10]:
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs
for model, fields in models.items():    
  print("model:",model)
  models[model] = MultiLabelClassificationModel(fields["model_type"], os.path.join(TRANSFORMERS_PATH, model), use_cuda=False)

model: mbert-cased
model: mbert-uncased
model: roberta
model: beto-cased
model: beto-uncased
model: distilbert-multi


In [11]:
import json
best_ensemble = {}
with open(os.path.join(EXP_PATH, "best-ensemble.json")) as json_file:
    best_ensemble = json.load(json_file)
print("----- BEST ENSEMBLE -----")
for field in ["name", "models", "metrics"]:
  print(f"{field}:", best_ensemble.get(field))

----- BEST ENSEMBLE -----
name: ensemble19
models: ['beto-cased', 'distilbert-multi']
metrics: {'accuracy': 0.95665, 'macro_f1': 0.88668, 'macro_precision': 0.95511, 'macro_recall': 0.83622, 'weighted_f1': 0.95303}


In [12]:
model_evaluation = {}
for model in best_ensemble.get("models"):  
  with open(os.path.join(os.path.join(EXP_PATH, model), "model-evaluation.json")) as json_file:
      model_evaluation[model] = json.load(json_file)

In [13]:
from sklearn.preprocessing import normalize
def vote(predictions, weighted=False, weights=None):
  votes = list()  
  for i in range(len(labels)):    
    curr_label_preds = [preds[i] for preds in predictions]    
    voting = sum(curr_label_preds * weights) if weighted else sum(curr_label_preds)/len(curr_label_preds)
    votes.append(0 if voting < 0.5 else 1)  
  return votes
test_predictions = list()
def predict_ensemble(ensemble_name, dataset_name, dataset, weighted=False):  
 for i in range(len(dataset.index)):
    predictions = list()
    ensemble_models = best_ensemble.get("models")    
    for model_name in ensemble_models:
      curr_model_outputs = model_evaluation[model_name].get(f"{dataset_name}_model_outputs")
      predictions.append(curr_model_outputs[i])
    weights = list()    
    if weighted:      
      f1_scores_list = [model_evaluation[model_name]["metrics"].get("weighted_f1")                        
                        for model_name in best_ensemble.get("models")]
      weights = normalize([f1_scores_list], norm="l1")[0]
    ensemble_pred = vote(predictions, weighted, weights)
    test_predictions.append(ensemble_pred)

In [14]:
for model_name in best_ensemble.get("models"):
  model_predictions, model_raw_outputs = models.get(model_name).predict(test["text"].tolist())
  model_evaluation[model_name]["test_model_outputs"] = model_raw_outputs
  model_evaluation[model_name]["test_predictions"] = model_predictions
predict_ensemble(best_ensemble.get("name"), "test", test, weighted=WEIGHTED)
n = 5
print(f"First {n} predictions:", test_predictions[:n])

3it [00:03,  1.25s/it]                                                                                                                                                                                                                  
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 182/182 [01:50<00:00,  1.64it/s]
3it [00:03,  1.16s/it]                                                                                                                                                                                                                  
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 182/182 [00:56<00:00,  3.24it/s]

First 5 predictions: [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


In [15]:
test_output = test[["id"]].rename(columns={"id": "id"})
for i in range(len(labels)):
  test_output[labels[i]] = [pred[i] for pred in test_predictions]
test_output.head(10)

,id,political,racial/ethnic,religious,gender/sexual,other
0,eng_0d46cf5c8e1f037159cfd6c9dcde03bb,0,0,0,0,0
1,eng_0676b42cc5e76f3a2b7cf3b562676fc1,0,0,0,0,0
2,eng_f3ce74f07a4cd9ee1926b87b40b78887,0,0,0,0,0
3,eng_9bb8d97251cc86cf5943c31be5f1b9a6,0,0,0,0,0
4,eng_755cacee0509beb4dc49f0fa1fd5d22f,0,0,0,0,0
5,eng_9e29b6877b966ac9b226e9bd016fa592,1,0,0,0,0
6,eng_3aa28369c1da093e29ee4559f4a33cdf,0,0,0,0,0
7,eng_349848e9dc4fced9790b7cc4d9c7fe75,0,0,0,0,0
8,eng_e071c8bea882732757fe7032b4e04ca5,0,0,0,0,0
9,eng_a930dd0488308b34fe63829d521a2fba,0,0,0,0,0


In [17]:
test_output.to_csv(os.path.join(OUTPUT, "results_test3.csv"), index=False)